In [ ]:
%matplotlib inline

In [ ]:
import uproot
import matplotlib.pyplot as plt

import vector
import awkward
import numpy as np
import glob

import fastjet

In [ ]:
#Remap various PDG-IDs to just photon, electron, muon, tau, charged hadron, neutral hadron
def map_pdgid_to_candid(pdgid, charge):
    if pdgid == 0:
        return 0

    #photon, electron, muon
    if pdgid in [22, 11, 13, 15]:
        return pdgid

    # charged hadron
    if abs(charge) > 0:
        return 211

    # neutral hadron
    return 130

In [ ]:
!ls -lrt /local/joosep/clic_edm4hep/

In [ ]:
files = list(glob.glob("/home/joosep/particleflow/fcc/p8_ee_ZH_Htautau_ecm380/*.root"))

In [ ]:
arrs = []
for fn in files[:10]:
    fi = uproot.open(fn)
    ev = fi["events"]
    this_file_arrs = ev.arrays(["MCParticles", "MergedRecoParticles"])
    
    idx0 = "RecoMCTruthLink#0/RecoMCTruthLink#0.index"
    idx1 = "RecoMCTruthLink#1/RecoMCTruthLink#1.index"

    idx_recoparticle = ev.arrays(idx0)[idx0]
    idx_mcparticle = ev.arrays(idx1)[idx1]
    
    #index in the MergedRecoParticles collection
    this_file_arrs["idx_reco"] = idx_recoparticle
    
    #index in the MCParticles collection
    this_file_arrs["idx_mc"] = idx_mcparticle
    
    arrs.append(this_file_arrs)
arrs = awkward.concatenate(arrs)

In [ ]:
#Compute 4-momentum of MC particles
mcp = arrs["MCParticles"]
mcp = awkward.Record({k.replace("MCParticles.", ""): mcp[k] for k in mcp.fields})
mc_p4 = vector.awk(awkward.zip({
    "mass": mcp["mass"],
    "x": mcp["momentum.x"],
    "y": mcp["momentum.y"],
    "z": mcp["momentum.z"]}))

In [ ]:
mc_p4

In [ ]:
# Get the matched reco and MC particles, flatten across all events, plot energy of reco vs. gen.

plt.figure(figsize=(5,5))
b = np.logspace(-1,3,100)
plt.hist2d(
    awkward.to_numpy(awkward.flatten(mc_p4.energy[arrs["idx_mc"]])),
    awkward.to_numpy(awkward.flatten(arrs["MergedRecoParticles"]["MergedRecoParticles.energy"][arrs["idx_reco"]])),
    bins=(b,b),
    cmap="Blues"
);
plt.xscale("log")
plt.yscale("log")

In [ ]:
#Get the taus (status=2 is unstable/decaying)
mask_taus = (np.abs(mcp["PDG"])==15) & (mcp["generatorStatus"]==2)

In [ ]:
mc_p4.energy[mask_taus]

In [ ]:
b = np.logspace(-2,3,101)

plt.hist(
    awkward.flatten(mc_p4.energy[mask_taus]),
    bins=b,
    label="15",
    histtype="step",
    lw=2
)
plt.xscale("log")
plt.xlabel("Energy [GeV]")
plt.ylabel("Number of particles / bin")
plt.title("Generator taus")
plt.legend()
plt.xscale("log")

In [ ]:
#Get the stable pythia particles
msk = mcp["generatorStatus"]==1
mc_pid = awkward.flatten(mcp.PDG[msk])
mc_charge = awkward.flatten(mcp.charge[msk])
mc_energy = awkward.flatten(mc_p4.energy[msk])

#map PDGID to candidate ID (similar labels as for PF)
mc_candid = np.array([
    map_pdgid_to_candid(abs(pdgid), charge) for pdgid, charge in zip(mc_pid, mc_charge)
])

b = np.logspace(-2,3,101)
for pid in np.unique(mc_candid):
    plt.hist(
        mc_energy[mc_candid==pid],
        bins=b,
        histtype="step",
        lw=2,
        label=pid
    );
plt.legend()
plt.xscale("log")
plt.xlabel("Energy [GeV]")
plt.ylabel("Number of particles / bin")
plt.title("Stable gen particles")

In [ ]:
#Prepare 4-momentum of reco particles
mrp = arrs["MergedRecoParticles"]
mrp = awkward.Record({k.replace("MergedRecoParticles.", ""): mrp[k] for k in mrp.fields})
reco_p4 = vector.awk(awkward.zip({
    "mass": mrp["mass"],
    "x": mrp["momentum.x"],
    "y": mrp["momentum.y"],
    "z": mrp["momentum.z"]}))

In [ ]:
#Remove type=0 reco particles (not sure what they are, but they are never matched to genparticles)
msk = mrp["type"]!=0

reco_pid = awkward.flatten(mrp["type"][msk])
reco_charge = awkward.flatten(mrp.charge[msk])
reco_energy = awkward.flatten(reco_p4.energy[msk])

reco_candid = np.array([
    map_pdgid_to_candid(abs(pdgid), charge) for pdgid, charge in zip(reco_pid, reco_charge)
])


b = np.logspace(-2,3,101)
for pid in np.unique(reco_candid):
    plt.hist(
        reco_energy[reco_candid==pid],
        bins=b,
        histtype="step",
        lw=2,
        label=pid
    );
plt.legend()
plt.xscale("log")
plt.xlabel("Energy [GeV]")
plt.ylabel("Number of particles / bin")
plt.title("reco particles")

In [ ]:
#This is how you can check what collection corresponds to what collectionID
collectionIDs = {k: v for k, v in
    zip(fi.get("metadata").arrays("CollectionIDs")["CollectionIDs"]["m_names"][0],
    fi.get("metadata").arrays("CollectionIDs")["CollectionIDs"]["m_collectionIDs"][0])}
collectionIDs_reverse = {v: k for k, v in collectionIDs.items()}

In [ ]:
#Cluster AK4 jets from PF particles with min pt 2 GeV
jetdef = fastjet.JetDefinition(fastjet.antikt_algorithm, 0.4)
cluster = fastjet.ClusterSequence(reco_p4.to_xyzt(), jetdef)
jets = vector.awk(cluster.inclusive_jets(min_pt=2.0))

In [ ]:
plt.hist(awkward.flatten(jets.pt), bins=100);

In [ ]:
#Get the PF particle indices in each jet
constituent_idx = cluster.constituent_index(min_pt=2.0)
constituent_idx

In [ ]:
constituent_idx.layout

In [ ]:
p4_flat = reco_p4[awkward.flatten(constituent_idx, axis=-1)]
num_ptcls_per_jet = awkward.num(constituent_idx, axis=-1)
ret = awkward.from_iter(
    [awkward.unflatten(p4_flat[i], num_ptcls_per_jet[i], axis=-1) for i in range(len(num_ptcls_per_jet))])

In [ ]:
ret2 = vector.awk(awkward.zip({"x": ret.x, "y": ret.y, "z": ret.z, "t": ret.tau}))

In [ ]:
ret2

In [ ]:
ret2[0][0]

In [ ]:
constituent_idx[0]